# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [4]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

In [4]:
# Определяем максимальную длину ключа
max_key = max(len(key) for key in obj.keys())

for key, value in obj.items():
    # < указывает на выравнивание по левому краю
    # {max_key_length} определяет ширину поля, в котором будет выведен ключ 
    print(f'{str(key):<{max_key_length}} = {value}')


home_page = https://github.com/pypa/sampleproject
keywords  = sample setuptools development
license   = MIT


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [5]:
import pandas as pd

obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

In [19]:
import re

p = re.compile(r'[0-9]{2}[-][0-9]')
for i in obj:
    res = re.search(p, i)
    print(i.split()[0], res.group())

Евгения 19-1
Илья 20-4
Анна 20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [25]:
from razdel import tokenize

t = 'Написать регулярное выражение,которое позволит найти номера групп студентов.'
res = list(tokenize(t))
res

[Substring(0, 8, 'Написать'),
 Substring(9, 19, 'регулярное'),
 Substring(20, 29, 'выражение'),
 Substring(29, 30, ','),
 Substring(30, 37, 'которое'),
 Substring(38, 46, 'позволит'),
 Substring(47, 52, 'найти'),
 Substring(53, 59, 'номера'),
 Substring(60, 65, 'групп'),
 Substring(66, 75, 'студентов'),
 Substring(75, 76, '.')]

In [35]:
res = re.split(r'[., ]+', t)
res

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов',
 '']

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [52]:
recipes = pd.read_csv('recipes_sample.csv')
random_recipes = recipes.sample(5)

table = "| {:^10} | {:^10} |\n|".format("id", "minutes") + "-"*25 + "|\n"

# Добавляем информацию о каждом рецепте в таблицу
for index, row in random_recipes.iterrows():
    table += "| {:^10} | {:^10} |\n".format(str(row['id']), str(row['minutes']))

# Выводим таблицу
print(table)

|     id     |  minutes   |
|-------------------------|
|   399575   |     5      |
|   89481    |     30     |
|   359154   |     40     |
|   208451   |     30     |
|   129136   |     35     |



2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [70]:
from bs4 import BeautifulSoup
with open('steps_sample.xml', 'r') as f:
  r = BeautifulSoup(f, 'xml')

id = []
st = []
for recipe in r.recipes.find_all('recipe'):
    id += [i.next for i in recipe.find_all('id')]
    st.append([step.next for step in recipe.steps.find_all('step')])

steps = {i:j for i,j in zip(id, st)}

def show_info(name, steps, minutes, author_id):
    # Формирование строки с описанием
    description = f'"{name}"\n\n'
    for i, step in enumerate(steps, start=1):
        description += f"{i}. {step}\n"
    description += "----------\n"
    description += f"Автор: {author_id}\n"
    description += f"Среднее время приготовления: {minutes} минут"
    
    return description

# Шаг 4: Вызов функции для рецепта с id 170895 и вывод результата
recipe_id = 170895
recipe = recipes[recipes['id'] == recipe_id].iloc[0]
name = recipe['name']
author_id = recipe['contributor_id']
minutes = recipe['minutes']
step = steps[str(recipe_id)]
info = show_info(name, step, minutes, author_id)
print(info)
    

"leeks and parsnips  sauteed or creamed"

1. clean the leeks and discard the dark green portions
2. cut the leeks lengthwise then into one-inch pieces
3. melt the butter in a medium skillet , med
4. heat
5. add the garlic and fry 'til fragrant
6. add leeks and fry until the leeks are tender , about 6-minutes
7. meanwhile , peel and chunk the parsnips into one-inch pieces
8. place in a steaming basket and steam 'til they are as tender as you prefer
9. i like them fork-tender
10. drain parsnips and add to the skillet with the leeks
11. add salt and pepper
12. gently sautee together for 5-minutes
13. at this point you can serve it , or continue on and cream it:
14. in a jar with a screw top , add the half-n-half and arrowroot
15. shake 'til blended
16. turn heat to low under the leeks and parsnips
17. pour in the arrowroot mixture , stirring gently as you pour
18. if too thick , gradually add the water
19. let simmer for a couple of minutes
20. taste to adjust seasoning , probably an addi

In [60]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

AssertionError: 

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [73]:
st = steps['25082']
for i in st:
    if re.findall(r'\d+ (?:hour[s]?|minute[s]?)', i ): 
        print(re.findall(r'\d+ (?:hour[s]?|minute[s]?)', i ))

['20 minutes']
['10 minutes']
['2 hours']
['10 minutes']
['20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [74]:
s = recipes.dropna()[recipes.dropna().description.str.contains('^this[A-Za-z0-9.,?! ]+, ?but')]['description']
print(len(s))
for i in range(3):
  print(s.iloc[i])

155
this cookie is made without flour and without sweetened condensed milk.  light and chewy.  i prefer using unsweetened coconut, but sweetened is fine.  this was my grandmother's recipe... probably from better homes and gardens.  they are flat, not the traditional mounded cookie.  perfect for dipping!
this is a moist, buttery apple cake.  the dough is best made the night before, put in the fridge and then filled and baked the next morning.
this is a very refreshing, nice summer dessert. very simple to prepare. you can use any pie filling that you wish, but we like lemon the best...it makes a nice contrast with the sweetness of the cream cheese mixture.


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [80]:
step = steps['72367']
for i in step:
    i = re.sub(r'\s*/\s*', r'/', i)
    print(i)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [84]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [96]:
st = []
for recipe in r.recipes.find_all('recipe'):
  st.append([step.next for step in recipe.steps.find_all('step')])
    
l = nltk.word_tokenize(' '.join([' '.join(i) for i in st])) 
print(len(set(i.lower() for i in l if i.isalpha())))

14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [97]:
recipes['n_sent'] = recipes['description'].apply(lambda i: len(sent_tokenize(str(i))))
recipes.nlargest(5, ['n_sent'])


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_sent
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN,76
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0,27
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0,24
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN,23
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN,23


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [101]:
name = recipes[recipes['id'] == 241106]['name'].values
name

array(['eggplant steaks with chickpeas  feta cheese and black olives'],
      dtype=object)

In [102]:
sent = 'eggplant steaks with chickpeas  feta cheese and black olives'
def sent_tag(sent):
  w = nltk.tag.pos_tag(nltk.word_tokenize(sent))
  up, down = '', ''
  for i in w:
    up += f' {i[1]: ^{len(i[0])}}'
    down += i[0]+' '
  return up+'\n'+down
print(sent_tag(sent))

    JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
